In [2]:
!pip install nibabel numpy pandas tqdm

In [3]:
# stats_nifti_batch.py
import glob
import os
from pathlib import Path

import nibabel as nib
import numpy as np
import pandas as pd
from tqdm import tqdm   # 進度條，可選

def compute_stats(nifti_path: str) -> dict:
    """讀取單一 NIfTI 檔並回傳統計量。"""
    img = nib.load(nifti_path)
    data = img.get_fdata()
    data = data[~np.isnan(data)]         # 排除 NaN

    return {
        "File": Path(nifti_path).name,
        "Min": float(np.min(data)),
        "Max": float(np.max(data)),
        "Mean": float(np.mean(data)),
        "Std": float(np.std(data, ddof=0)),
        "99th Percentile": float(np.percentile(data, 99)),
    }

def main():
    # ===== 1. 取得檔案清單 =====
    # 方法 A：資料夾遞迴搜尋
    nifti_dir = "/Users/kungtzuyun/Lab/ACTI/Code/trial_analysis/behavior_codes/group_avg_maps"   # ← 修改路徑
    pattern = "**/*.nii"                       # 或 "**/*.nii.gz"
    nii_files = sorted(glob.glob(os.path.join(nifti_dir, pattern), recursive=True))

    # 方法 B：若已有明確檔案清單
    # nii_files = ["/path/f1.nii", "/path/f2.nii", ...]   # 130 張

    if len(nii_files) == 0:
        raise FileNotFoundError("找不到任何 .nii 檔，請檢查路徑或副檔名！")

    # ===== 2. 批次計算 =====
    stats = []
    for f in tqdm(nii_files, desc="計算統計中"):
        stats.append(compute_stats(f))

    # ===== 3. 匯出或顯示 =====
    df = pd.DataFrame(stats)
    print(df)                  # 直接印出
    out_csv = Path(nifti_dir).joinpath("nifti_intensity_stats.csv")
    df.to_csv(out_csv, index=False)
    print(f"\n✅ 已將結果存成 CSV：{out_csv}")

if __name__ == "__main__":
    main()

計算統計中: 100%|██████████| 130/130 [00:00<00:00, 370.16it/s]

                       File         Min         Max       Mean        Std  \
0    group_avg_con_0001.nii -126.320541  160.983505  -3.074138  15.201924   
1    group_avg_con_0002.nii  -75.709137   81.843231  -0.428828  11.197278   
2    group_avg_con_0003.nii   -1.911565   10.313158   1.264759   1.335028   
3    group_avg_con_0004.nii   -7.432490    7.094760   0.164478   1.057425   
4    group_avg_con_0005.nii   -6.665852    4.939939   0.046726   1.050410   
..                      ...         ...         ...        ...        ...   
125  group_avg_con_0126.nii -120.196457  343.890778   3.254651  24.700736   
126  group_avg_con_0127.nii -126.461098   99.387901   3.725861  21.084842   
127  group_avg_con_0128.nii -205.958130  106.322838  12.270223  19.854831   
128  group_avg_con_0129.nii  -61.457977  134.290939  13.177217  20.099246   
129  group_avg_con_0130.nii  -80.064796  168.045227  13.833160  20.697972   

     99th Percentile  
0          49.556226  
1          26.925643  
2     